# Description du problème 

## Contexte 
Le marché américain des actions est l'un des plus liquides au monde, et fournit de nombreuses opportunités d'investissement. Les deux dernières heures de la session, entre 14H et 16H, sont les plus liquides de toutes. <br>
<br>
Cette période est la préférée pour acheter ou vendre une grande quantité d'actifs, car les conditions de marché sont alors meilleures (coûts de transaction plus bas, volatilité moindre, …). C'est pourquoi estimer à l'avance le comportement d'un actif sur cette période permet d'optimiser l'ensemble de son portefeuille. <br>
<br>

## But 
Le but de ce challenge est d'estimer la direction du prix d'une action durant les deux dernières heures de trading, connaissant le comportement du début de la journée. <br>
<br>
Pour éviter le bruit habituel des mouvements financiers, nous ne considérons que trois directions : <br>

- une baisse franche du prix ;
- une évolution mineure, quelque soit le sens ;
- une hausse franche du prix.

## Description des données
Nous fournissons les évolutions de prix (rendements) avec une granularité de 5 minutes, ce qui aboutit à 53 valeurs sur ces 4,5 heures, pour chaque jour et action. <br>
<br>
Comme ces rendements sont faibles, nous les exprimons en point de base (bps), c'est-à-dire $ \space \frac{P_{t+5 mn} - P_t}{P_t} . 10^{-4}$. <br>
<br>
Colonnes : <br>

- 'ID', l'identifiant unique de cette ligne ;
- 'day', l'identifiant du jour concerné (non unique dans le jeu de données) ;
- 'equity', l'identifiant de l'action concernée (non unique dans le jeu de donnée) ;
- '$r_0$' = $ \space \frac{P_{9:35} - P_{9:30}}{P_{9:30}} . 10^{-4}$ 
- ... 
- '$r_{52}$' = $ \space \frac{P_{14:00} - P_{13:55}}{P_{13:55}} . 10^{-4}$ 

Pour réduire la difficulté de la tâche, la prédiction se limite à estimer la direction du rendement des deux dernières heures, groupée en 3 cas : <br>

- $-1$ si $ \space \frac{P_{16 \text{PM}} - P_{14 \text{PM}}}{P_{14 \text{PM}}} . 10^{-4}$ est inférieur à $-25 \text{bps}$; 
- $0$ si ce rendement est compris entre $-25 \text{bps}$ et $25 \text{bps}$;
- $1$ s'il est supérieur à $25 \text{bps}$ 

Le fichier de prédiction comprend les colonnes :

- 'ID', l'identifiant unique la ligne prédite ;
- 'reod', la classe estimée du rendement de la fin de session, incluse dans {$-1,0,1$}

Il est à noter que pour éviter toute fuite d'information, les jeux d'entrainement et de test ne comportent pas d'actions ni de jours en commun. Les actions des deux datasets ont cependant exactement les mêmes caractéristiques financières, afin de pouvoir prédire leurs comportements. Néanmoins, les deux périodes concernées sont totalement différentes, afin de correspondre à un vrai travail de prédiction, sur des données réelles d'un monde dont les caractéristiques peuvent changer.